Field Interpolation
==============

Given a set of samples of an unkown function, _estimate_ a function $f$


In [ ]:
import seaborn as sns
sns.set_theme()
sns.set(style='darkgrid', context='talk', palette='Pastel1')


In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 5]

$$f(0) = -1$$
$$f(1) = 0.2$$
$$f(2) = 0.9$$
$$f(3) = 2.1$$

Estimate a line, $f(x) = kx + m$, using linear last squares

$$-1 = k*0 + m$$
$$0.2 = k*1 + m$$
$$0.9 = k*2 + m$$
$$2.1 = k*3 + m$$

Or just

$$
\begin{bmatrix}
0 & 1 \\
1 & 1 \\
2 & 1 \\
3 & 1 \\
\end{bmatrix}
\begin{bmatrix} k \\ m \end{bmatrix}
= 
\begin{bmatrix}
-1 \\ 0.2 \\ 0.9 \\ 2.1 \\
\end{bmatrix}$$
$$A\begin{bmatrix} k \\ m \end{bmatrix} = \bf{b}$$

In [ ]:
import numpy as np

# sampled points
x = np.array([0, 1, 2, 3])
y = np.array([-1, 0.2, 0.9, 2.1])

A = np.vstack([x, np.ones(x.shape)]).T
k, m = np.linalg.lstsq(A, y, rcond=None)[0]

In [ ]:
def plot(x0, y0, x1, y1):
    plt.plot(x0, y0, 'o', label='Original data')
    plt.plot(x1, y1, label='Fitted line')
    plt.legend()
    plt.show()

plot(x, y, x, k*x + m)

### Lattice model
Instead of using a line, lets use the _values_ at specific lattice points as the model. We use a finite grid with a high and low limit. The downside is we can only evaluate it within the bounds, whereas the line model could be evaluated everywhere.

$$
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
\end{bmatrix}
\begin{bmatrix}
x_0 \\ x_1 \\ x_2 \\ x_3
\end{bmatrix}
=
\begin{bmatrix}
-1 \\ 0.2 \\ 0.9 \\ 2.1 \\
\end{bmatrix}
$$

$$A\bf{x} = \bf{b}$$



In [ ]:
plot(x, y, x, y)

But what if we have more points? And that are not exactly _on_ the lattice grid?

$$f(0) = -1$$
$$f(1) = 0.2$$
$$f(2) = 0.9$$
$$f(2.2) = 1$$
$$f(3) = 2.1$$

Then we do linear interpolation ("lerp"). The closest lattice points are 2 and 3.

$$0.8 \cdot f(2) + 0.2 \cdot f(3) = 1$$

In [ ]:
EPSILON = 1e-8

def build_A(xx, n):
    def rows():
        for f, i in zip(*np.modf(x)):
            row = np.zeros(n)
            if 1 - f > EPSILON:
                row[int(i)] = 1 - f
            if f > EPSILON:
                row[int(i) + 1] = f
            yield row
    return np.array(list(rows()))
    
x = np.array([0, 1, 2, 2.2, 3])
y = np.array([-1, 0.2, 0.9, 1, 2.1])

A = build_A(x, 4)
A

In [ ]:
xg = np.linalg.lstsq(A, y, rcond=None)[0]
plot(x, y, np.arange(len(xg)), xg)